In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.misc
import pandas as pd
import os

# mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

mb_size = 16
Z_dim = 100
X_dim = 116412
y_dim = 23
h_dim = 128

In [ ]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        # plt.imshow(sample.reshape(178, 218), cmap='Greys_r')
        plt.imshow(sample.reshape(218, 178, 3))

    return fig

In [ ]:
def load_train_data():
    cols = ["Arched_Eyebrows", "Bags_Under_Eyes", "Bangs", "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Brown_Hair",
       "Bushy_Eyebrows","Eyeglasses","Heavy_Makeup","High_Cheekbones","Male", "Mouth_Slightly_Open","Narrow_Eyes",
       "No_Beard","Oval_Face","Pointy_Nose","Smiling","Straight_Hair","Wavy_Hair","Wearing_Hat","Young"]

    attr = pd.read_csv('data/list_attr_celeba.csv', delim_whitespace=True, skiprows=1, usecols=cols)

    attr = attr.values[:10000]
    print "shape of attr: {}".format(attr.shape)
   
    X = []

    for i in xrange(10000):
        X_ = scipy.misc.imread('data/img_align_celeba/{:06d}.jpg'.format(i + 1))
        X_ = X_.reshape(218 * 178 *3)
        #X_ = np.concatenate([X_, attr[i]])
        X.append(X_)

    X = np.array(X)
   
    print "shape of one image: {}".format(X[0].shape)
    print "shape of X: {}".format(X.shape)
    return X, attr


# Load cifar-10 data
celeb_data, attr = load_train_data()# / 255.0
celeb_data = celeb_data / 255.0

# x = tf.placeholder(tf.float32, shape=[None, 116435])
# y = tf.placeholder(tf.float32, shape=[None, 23])
    

In [ ]:
def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

def conv2d(input, output_shape, k_h=5, k_w=5, stddev=0.02, name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input.get_shape()[-1], output_shape],
                initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input, w, strides=[1, 2, 2, 1], padding='SAME')
        biases = tf.get_variable('biases', [output_shape], initializer=tf.constant_initializer(0.0))
        conv = lrelu(tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape()))
    return conv

def max_pool(input, kernel_size, stride):
    ksize = [1, kernel_size, kernel_size, 1]
    strides = [1, stride, stride, 1]
    return tf.nn.max_pool(input, ksize=ksize, strides=strides, padding='SAME')

In [ ]:
""" Discriminator Net model """
X = tf.placeholder(tf.float32, shape=[mb_size, X_dim], name='real_images')
y = tf.placeholder(tf.float32, shape=[mb_size, y_dim])

output_dim = 64

def discriminator(x, y, scope='discriminator', reuse=True):
    print ("discriminator") 
    print ("shape x =")
    print (x.shape)
    print ("shape y =")
    print (y.shape)
    
    with tf.variable_scope(scope, reuse=reuse) as scope:
        x = tf.reshape(x, [tf.shape(x)[0], 178, 218, 3])

        D_h1 = conv2d(x, output_dim, name='conv1')  
        D_h2 = conv2d(D_h1, output_dim * 2, name='conv2')
        D_h3 = conv2d(D_h2, 5, 2, output_dim * 4, name='conv3')
        D_h4 = tf.nn.relu(conv2d(D_h3, 5, 2, output_dim * 8, name='conv4'))
        
        print "D_h1 shape: {}".format(D_h1.shape)
        print "D_h2 shape: {}".format(D_h2.shape)
        print "D_h3 shape: {}".format(D_h3.shape)
        print "D_h4 shape: {}".format(D_h4.shape)

        #flatten
        flattened_image = tf.contrib.layers.flatten(D_h4)
        
        #add atributes
        image_attributes = tf.concat(axis=1, values=[flattened_image, y])

        #fully connected layer
        fc1 = tf.contrib.layers.fully_connected(image_attributes, 1, activation_fn=None)
        D_prob = tf.nn.sigmoid(fc1)
        
    return D_prob, fc1

In [ ]:
""" Generator Net model """
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim + y_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim])) 

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z, y):
    print ("generator") 
    print ("shape z =")
    print (z.shape)
    print ("shape y =")
    print (y.shape)
    inputs = tf.concat(axis=1, values=[z, y])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
#     G_h1 = tf.nn.relu(tf.nn.conv2d_transpose(inputs, 5, 2, 32))
#     G_log_prob = tf.nn.conv2d_transpose(G_h1, G_W2)

    return G_prob

In [ ]:

G_sample = generator(Z, y)
D_real, D_logit_real = discriminator(X, y, reuse=False)
D_fake, D_logit_fake = discriminator(G_sample, y, reuse=True)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

In [ ]:
features = [("Arched_Eyebrows", -1), ("Bags_Under_Eyes", -1), ("Bangs", 1), ("Big_Lips", -1), ("Big_Nose", -1),
            ("Black_Hair", -1), ("Blond_Hair", 1), ("Brown_Hair", -1), ("Bushy_Eyebrows", -1), ("Eyeglasses", -1),
            ("Heavy_Makeup", -1), ("High_Cheekbones", 1), ("Male", 1), ("Mouth_Slightly_Open", -1), ("Narrow_Eyes", 1),
            ("No_Beard", -1), ("Oval_Face", 1), ("Pointy_Nose", -1), ("Smiling", 1), 
            ("Straight_Hair", 1), ("Wavy_Hair", -1), ("Wearing_Hat", -1), ("Young", 1)]
features = map(lambda x: x[1], features)
print features

In [ ]:
import tensorflow.contrib.slim as slim

all_var = tf.trainable_variables()
theta_D = [v for v in all_var]

print("********* d_var ********** ")
slim.model_analyzer.analyze_vars(theta_D, print_info=True)
print("********* g_var ********** ")
slim.model_analyzer.analyze_vars(theta_G, print_info=True)


def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


learning_rate = 1e-4

D_solver = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(G_loss, var_list=theta_G)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out_discriminator/'):
    os.makedirs('out_discriminator/')
    
    
i = 0
update_ratio = 5
G_loss_curr = None

D_losses = []
G_losses = []

for it in range(1000):
    if it % 10 == 0:
        n_sample = 16

        Z_sample = sample_Z(n_sample, Z_dim)
        # y_sample = np.zeros(shape=[n_sample, y_dim])
        # y_sample[:, 7] = 1
        # y_sample = np.array([-1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1])
        # features = [-1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1]
        y_sample = np.array(features)
        y_sample = np.tile(y_sample, (16, 1))
        
        samples = sess.run(G_sample, feed_dict={Z: Z_sample, y: y_sample})

        fig = plot(samples)
        plt.savefig('out_discriminator/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
    
    X_mb, y_mb = next_batch(mb_size, celeb_data, attr)

    Z_sample = sample_Z(mb_size, Z_dim)

    if it % update_ratio == 0:
        _, D_loss_curr, D_real_cur, D_fake_cur = sess.run([D_solver, D_loss, D_real, D_fake], feed_dict={X: X_mb, Z: Z_sample, y:y_mb})
        D_losses.append(D_loss_curr)
    else:
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: Z_sample, y:y_mb})
        G_losses.append(G_loss_curr)

    if it % 10 == 0:
        print('Iter: {}'.format(it))
        print('D real mean: {}'. format(D_real_cur.mean()))
        print('D fake mean: {}'. format(D_fake_cur.mean()))
        print('D loss: {:.4}'. format(D_loss_curr))
        
        print('G_loss: {:.4}'.format(G_loss_curr))
        
        print()

plt.subplot(2, 1, 1)
plt.title('D_loss')
D_losses = D_losses[1::10] # sparse the curve a bit
plt.plot(D_losses, '-o')
plt.xlabel('Iteration')

plt.subplot(2, 1, 2)
plt.title('G_loss')
G_losses = G_losses[1::40]
plt.plot(G_losses, '-o')
plt.xlabel('Iteration')

plt.show()